In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Embedding
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
epochs=4
batch_size=128
n_dim=64
n_unique_words=5000
n_words_skip=50
max_review_length=100
pad_type=trunc_type='pre' 

n_dense=64
dropout=0.5

In [ ]:
#Loading Data
(xtrain,ytrain),(xvalid,yvalid)= imdb.load_data(num_words=n_unique_words,skip_top=n_words_skip)

In [ ]:
xtrain[0]  # 0 is reserved dfor padding
# 2 is uknown word

In [ ]:
xtrain[0:5]

In [ ]:
for i in xtrain[0:5]:
    print(len(i))

In [ ]:
ytrain[0:6]

In [ ]:
# Restoring words from index
word_index=keras.datasets.imdb.get_word_index()
word_index={k:(v+3) for k,v in word_index.items()}
# we are inserting new words into word index so we are increasing indexes of words by 3
word_index['PAD']=0
word_index['START']=1
word_index['UNK']=2

In [ ]:
#flipping words and keys
index_word={v:k for k,v in word_index.items()}


In [ ]:
index_word

In [ ]:
" ".join(index_word[id] for id in xtrain[0])

In [ ]:
#Incase we want to load all words 
(all_xtrain,_),(all_test,_)=imdb.load_data()

In [ ]:
" ".join(index_word[id] for id in all_xtrain[0])

In [ ]:
xtrain1=pad_sequences(xtrain,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)

In [ ]:
xvalid1=pad_sequences(xvalid,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)

In [ ]:
[len(i) for i in xtrain1[0:6]]  # all are of same length now

In [ ]:
# Design NN architecture
model=Sequential()
model.add(Embedding(n_unique_words,n_dim,input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(xtrain1,ytrain,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(xvalid1,yvalid))

In [ ]:
y_hat=model.predict_proba(xvalid1)

In [ ]:
y_hat[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(y_hat)

In [ ]:
auc_cur=roc_auc_score(yvalid,y_hat)*100.0

In [ ]:
auc_cur

In [ ]:
#Looking individual results
float_yhat=[]
for y in y_hat:
    float_yhat.append(y[0])

In [ ]:
ydf=pd.DataFrame(list(zip(float_yhat,yvalid)),columns=['yhat','y'])

In [ ]:
ydf.head()

In [ ]:
" ".join(index_word[i] for i in all_test[3])
# some bad words too in this example

In [ ]:
#checking for examples where model predicted wrong
ydf[(ydf.y==0) & (ydf.yhat>0.9)].head(10)

In [ ]:
#checking 247
" ".join(index_word[i] for i in all_test[156])